In [10]:
import pandas as pd
import os
import numpy as np
import shutil
import re
import sklearn
import matplotlib.pyplot as plt
import pyreadr
import os
#import unidecodeall
#import xgboost as xgb

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    cohen_kappa_score,
    classification_report,
    accuracy_score,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import TweetTokenizer

from datasets import load_dataset, Dataset
from joblib import dump, load

from spanish_nlp import preprocess
from spanish_nlp import augmentation
from spanish_nlp.utils.stopwords import extended_stopwords

In [2]:
# Lee archivo csv con variables exógenas
ciuo_df = pd.read_csv("data_in/ciuo08_v8.csv",encoding='utf-8', index_col=0)
ciuo_df

,text_id,texto,clase,clase1
0,1.000000e+00,enfermero clinico atencion directas pacientes ...,22.0,2
1,2.000000e+00,enfermera clinica administracion atencion dire...,22.0,2
2,4.000000e+00,profesora ed fisica realizar clases ed fisica ...,23.0,2
3,5.000000e+00,cuidadora niño baña cuida comida niño años,53.0,5
4,7.000000e+00,vendedores almacen vende compra mercaderia,52.0,5
...,...,...,...,...
69634,1.329554e+11,operador de linea matanza de salmones para aba...,93.0,9
69635,1.329561e+11,assora de inversiones asesorias a clientes t...,33.0,3
69636,1.329618e+11,quimico farmacia realiza fabricacion de pro...,22.0,2
69637,1.329638e+11,"operario agricola opera tractor, recogedora ...",83.0,8


In [3]:
caenes_df = pd.read_csv("data_in/caenes.csv",encoding='latin1',index_col=0)
caenes_df.dropna(inplace=True)
caenes_df

,b14_glosa,cise,cine,b1_1_rev4,b14_1_rev4cl_caenes,b14_2d,texto,id
0,ELABORACION DE JUGOS NATURALES Y SANEWICHEROS,1.0,5.0,1.0,9.0,56,ELABORACION DE JUGOS NATURALES Y SANEWICHEROS,1
1,ELABORACION DE JUGOS NATURALES Y SANDWICHES,3.0,5.0,3.0,9.0,56,ELABORACION DE JUGOS NATURALES Y SANDWICHES,2
2,ELABORACIOND E JUGOS NATURALES Y SANDWICHES,2.0,6.0,4.0,9.0,56,ELABORACIOND E JUGOS NATURALES Y SANDWICHES,3
3,"SERVICIO MANTENCION DE SOLDADURA Y CALDERIA, M...",3.0,7.0,2.0,3.0,33,"SERVICIO MANTENCION DE SOLDADURA Y CALDERIA, M...",4
4,RECTIFICACION DE MOTORES CAMION DE ALTO TONELAJE,3.0,5.0,3.0,3.0,33,RECTIFICACION DE MOTORES CAMION DE ALTO TONELAJE,5
...,...,...,...,...,...,...,...,...
184262,ELABORACION DE PAN Y PASTELES,3.0,5.0,7.0,3.0,10,ELABORACION DE PAN Y PASTELES,184263
184263,DISTRIBUIDORA Y VENTA DE GAS NATURAL A DOMICILIO,3.0,5.0,8.0,7.0,48,DISTRIBUIDORA Y VENTA DE GAS NATURAL A DOMICILIO,184264
184264,"VENTA DE ABARROTES, CONSERVAS, UTILES DE ASEO ...",3.0,5.0,9.0,7.0,48,"VENTA DE ABARROTES, CONSERVAS, UTILES DE ASEO ...",184265
184265,SERVICIOS DE CONSTRUCCION Y REPARACION DE VE...,3.0,4.0,7.0,6.0,41,SERVICIOS DE CONSTRUCCION Y REPARACION DE VE...,184266


In [4]:
sp = preprocess.SpanishPreprocess(
        lower=True,
        remove_url=False,
        remove_hashtags=False,
        split_hashtags=False,
        normalize_breaklines=True,
        remove_emoticons=False,
        remove_emojis=False,
        convert_emoticons=False,
        convert_emojis=False,
        normalize_inclusive_language=False,
        reduce_spam=False,
        remove_reduplications=False,
        remove_vowels_accents=False,
        remove_multiple_spaces=True,
        remove_punctuation=True,
        remove_unprintable=False,
        remove_numbers=False,
        remove_stopwords=True,
        stopwords_list='default',
        lemmatize=False,
        stem=False,
        remove_html_tags=False,
)

In [5]:
ciuo_df['texto'] = ciuo_df['texto'].apply(lambda texto: sp.transform(texto, debug = False))
# Obtén la lista de etiquetas únicas
labels_unique = ciuo_df['clase'].unique()

In [6]:
caenes_df['texto'] = caenes_df['texto'].apply(lambda texto: sp.transform(texto, debug = False))

In [7]:
def evaluate(predicted_probabilities, y_test, labels):
    # Importante: al transformar los arreglos de probabilidad a clases,
    # entregar el arreglo de clases aprendido por el clasificador.
    # (que comunmente, es distinto a ['normal', 'odio', 'incivilidad'])
    predicted_labels = [
        labels[np.argmax(item)] for item in predicted_probabilities
    ]
    kappa = round(cohen_kappa_score(y_test, predicted_labels), 3)
    print("Kappa:", kappa, end="\t")
    accuracy = round(accuracy_score(y_test, predicted_labels), 3)
    print("Accuracy:", accuracy)
    f1_macro = round(f1_score(y_test, predicted_labels, average='macro'), 3)
    print("F1 macro:", f1_macro)
    f1_micro = round(f1_score(y_test, predicted_labels, average='micro'), 3)
    print("F1 micro:", f1_micro)
    print("------------------------------------------------------\n")
    return np.array([kappa, accuracy, f1_macro, f1_micro])

In [8]:
def run(samples, labels, pipeline, version):
    """Creamos el pipeline y luego lo ejecutamos el pipeline sobre un dataset.
    Retorna el modelo ya entrenado mas sus labels asociadas y los scores obtenidos al evaluarlo."""

    # Dividimos el dataset en train y test, aún no se transforma de Strings a valores numéricos.
    X_train, X_test, y_train, y_test = train_test_split(
        samples,
        labels,
        shuffle=True,
        test_size=0.2,
        random_state=42,
    )

    print(f"# Datos de entrenamiento en dataset: {len(X_train)}")
    print(f"# Datos de testing en dataset: {len(X_test)}")

    # Entrenamos el clasificador (Ejecuta el entrenamiento sobre todo el pipeline).
    # En este caso el Bag of Words es el encargado de transformar de Strings a vectores numéricos.
    pipeline.fit(X_train, y_train)
    dump(pipeline, 'models/'+version+'.joblib') 
    # Predecimos las probabilidades de intensidad de cada elemento del set de prueba.
    predicted_probabilities = pipeline.predict_proba(X_test)
    y_pred = pipeline.predict(X_test)
    # Obtenemos el orden de las clases aprendidas.
    learned_labels = pipeline.classes_

    # Evaluamos:
    scores = evaluate(predicted_probabilities, y_test, learned_labels)
    return pipeline, learned_labels, scores, y_test, y_pred

In [9]:
def baseline_1d():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion( # Acá van las features
                    [
                        ("tfidf", TfidfVectorizer()),
                    ]
                ),
            ),
            ("SVM_sin_esc", SVC(C=0.75,kernel='linear',probability=True)),
        ]
    )

In [10]:
def baseline_2d():

    return Pipeline(
        [
            (
                "features",
                FeatureUnion( # Acá van las features
                    [
                        ("tfidf", TfidfVectorizer()),
                    ]
                ),
            ),
            ("SVM", SVC(C=1.0,kernel='linear',probability=True)),
        ]
    )

In [10]:
# Creamos el pipeline
pipeline = baseline_1d()

# Ejecutamos el pipeline sobre el dataset y guardamos el clasificador, las labels aprendidas y los scores obtenidos
classifier_1d, learned_labels_1d, scores_1d, y_test_1d, y_pred_1d = run(ciuo_df.texto, ciuo_df.clase1, pipeline, 'baseline_ciuo1d')

# Datos de entrenamiento en dataset: 55711
# Datos de testing en dataset: 13928
Kappa: 0.773	Accuracy: 0.804
F1 macro: 0.717
F1 micro: 0.804
------------------------------------------------------



In [10]:
# Creamos el pipeline
pipeline = baseline_2d()

# Ejecutamos el pipeline sobre el dataset y guardamos el clasificador, las labels aprendidas y los scores obtenidos
classifier_2d, learned_labels_2d, scores_2d, y_test_2d, y_pred_2d = run(ciuo_df.texto, ciuo_df.clase, pipeline,'baseline_ciuo2d')

# Datos de entrenamiento en dataset: 55711
# Datos de testing en dataset: 13928
Kappa: 0.763	Accuracy: 0.773
F1 macro: 0.701
F1 micro: 0.773
------------------------------------------------------



In [11]:
# Creamos el pipeline
pipeline = baseline_1d()	

# Ejecutamos el pipeline sobre el dataset y guardamos el clasificador, las labels aprendidas y los scores obtenidos
classifier_1d, learned_labels_1d, scores_1d, y_test_1d, y_pred_1d = run(caenes_df.texto, caenes_df.b14_1_rev4cl_caenes, pipeline, 'baseline_caenes1d')

# Datos de entrenamiento en dataset: 142224
# Datos de testing en dataset: 35557
Kappa: 0.934	Accuracy: 0.94
F1 macro: 0.891
F1 micro: 0.94
------------------------------------------------------



In [12]:
# Creamos el pipeline
pipeline = baseline_2d()

# Ejecutamos el pipeline sobre el dataset y guardamos el clasificador, las labels aprendidas y los scores obtenidos
classifier_2d, learned_labels_2d, scores_2d, y_test_2d, y_pred_2d = run(caenes_df.texto, caenes_df.b14_2d, pipeline,'baseline_caenes2d')

# Datos de entrenamiento en dataset: 142224
# Datos de testing en dataset: 35557
Kappa: 0.91	Accuracy: 0.916
F1 macro: 0.785
F1 micro: 0.916
------------------------------------------------------



In [11]:
def consolidar_clasificaciones(carpeta):
    # Obtener la lista de archivos CSV en el directorio
    archivos_csv = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.csv')]

    # Verificar si hay archivos CSV en el directorio
    if not archivos_csv:
        print("No se encontraron archivos CSV en el directorio proporcionado.")
        return

    # Inicializar un diccionario para almacenar los datos consolidados
    datos_consolidados = {}

    # Recorrer cada archivo CSV y consolidar los datos
    for archivo_csv in archivos_csv:
        ruta_archivo = os.path.join(carpeta, archivo_csv)
        # Leer el archivo CSV
        datos = pd.read_csv(ruta_archivo)
        # Iterar sobre cada fila del DataFrame
        for indice, fila in datos.iterrows():
            id_texto = fila['id']
            clase = fila['preds']
            probabilidad = fila['probs']
            modelo = archivo_csv

            # Verificar si el ID de texto ya está en el diccionario
            if id_texto in datos_consolidados:
                # Verificar si la probabilidad es mayor que la almacenada
                if probabilidad > datos_consolidados[id_texto]['prob']:
                    # Actualizar los datos consolidados
                    datos_consolidados[id_texto] = {'clase': clase, 'prob': probabilidad, 'modelo': modelo}
            else:
                # Agregar el ID de texto al diccionario
                datos_consolidados[id_texto] = {'clase': clase, 'prob': probabilidad, 'modelo': modelo}

    # Convertir el diccionario a un DataFrame
    df_consolidado = pd.DataFrame.from_dict(datos_consolidados, orient='index').reset_index()
    df_consolidado.columns = ['ID', 'clase', 'prob', 'modelo']

    # Obtener el nombre del modelo de la ruta
    nombre_modelo = os.path.basename(carpeta)

    # Guardar el DataFrame consolidado en un archivo CSV
    ruta_salida = os.path.join("data_out", f'clasificacion_{nombre_modelo}.csv')
    df_consolidado.to_csv(ruta_salida, index=False)
    
    print(f"Clasificación consolidada guardada en: {ruta_salida}")

In [12]:
from pathlib import Path
filepath = Path('data_out/ciuo1d/')
consolidar_clasificaciones(filepath)

Clasificación consolidada guardada en: data_out/clasificacion_ciuo1d.csv
